In [2]:
import pandas as pd
import string
from unidecode import unidecode
from fuzzywuzzy import process, fuzz
from Levenshtein import distance, jaro, jaro_winkler, ratio

# **Importing the data**

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
!mkdir -p "drive/MyDrive/analise de endereços"

In [5]:
enderecos_15a = pd.read_excel('/content/drive/MyDrive/analise de endereços/enderecos_processos_em_andamento_15a.xlsx', index_col=0)
enderecos_15a

,numeroDocumentoPrincipal,nome,logradouro,numero,complemento,bairro,cidade,estado,pais,cep
0,bf580da36dfed6ee3721f0d26d63159014842b4c96b5b2...,770d60b893806689ec588a7bd1bfa5c7b3ee36645026e1...,NaN,NaN,NaN,NaN,Maceió,NaN,Brasil,NaN
1,7cb5d4bbb25f8d3a3cdcec95c1e88a264c62818cd9f782...,f3ecb0a4377b630243172484e0d01c2520c6a5426f060e...,Rua do Comércio,181,"Edf. Banco Econômico, 2º andar",Centro,Maceió,NaN,Brasil,NaN
2,e6dba38b2a567498a44c3e7d8aa6caff9d668381fd3e06...,05e2ce4d3f63e7005bdb772d65ac3378d0626808d0a254...,Av. Fernandes Lima,3349,NaN,Farol,Maceió,NaN,Brasil,NaN
3,0330aa7a12b5910df2712c58d667fb0fdb9b08b6bec5cf...,4428d9764b8a978655af3a268216981b505bb1a944bb03...,"Av. Fernandes Lima,S/N, Km 06,",NaN,NaN,Farol,Maceió,NaN,Brasil,57057900.0
4,f3d81c1857f9bf4b3721645ef9e85c0a81043ce8e6a1a9...,224799ce270da8c15f18971a8fc8c507fc6359702b9451...,Av. da PAZ,21,Lot: Leão Irmãos,Centro,Maceió,NaN,Brasil,57020440.0
...,...,...,...,...,...,...,...,...,...,...
66856,e9b27803e549a9b5e69802ee55fe309ca33be671d966b0...,9e7f79bd4c7375a81f2dc8875829c4646984fc8b307862...,Teonilo Gama,430,NaN,Trapiche da Barra,Maceió,NaN,Brasil,57010384.0
66857,7d7513d9c1b2983bf40232039546912f4189dbcab47442...,20bc6700304dc43938f97b02791e6ae053d8a53e552dab...,Rua - Jangadeiros Alagoanos,01188,Sala 0801,Pajucara,Maceió,NaN,Brasil,57.0
66858,d7d06d824002fd4c9a556c25f97ba5ac6b31aa384f70cf...,b973036c61c1b4c0f4c5d901774950961a36eb246c3344...,Rua - Jangadeiros Alagoanos,01188,Sala 0801,Pajucara,Maceió,NaN,Brasil,57.0
66859,413c194c770b76e0ad2df6e86ed096a38fc42c56dea088...,f350137c9264596d55cfb6844dba28c53cd2bcb2e54340...,Rua - Jangadeiros Alagoanos,01188,Sala 0801,Pajucara,Maceió,NaN,Brasil,57.0


In [6]:
enderecos_dne = pd.read_csv('/content/drive/MyDrive/analise de endereços/enderecos_dne.csv',sep=';')
enderecos_dne.columns = ['logradouro','cep','bairro','cidade']
enderecos_dne

,logradouro,cep,bairro,cidade
0,R Costa Cavalcante,57305620,Zélia Barbosa Rocha,Arapiraca
1,Av Jorge Barros,57063000,Santa Amélia,Maceió
2,Av Jorge Barros,57063005,Santa Amélia,Maceió
3,Av Jorge Barros,57071610,Clima Bom,Maceió
4,Q D,57048031,Antares,Maceió
...,...,...,...,...
10706,R Tucuruvi,57043495,Feitosa,Maceió
10707,R L1 M1,57072732,Cidade Universitária,Maceió
10708,Q L,57072302,Cidade Universitária,Maceió
10709,R Cel Ataíde de Oliveira,57313710,Brasília,Arapiraca


# **Pre-analysis**

Before doing some analysis, we are going to treat the data, removing some special characters and standardizing strings.

In [7]:
def normalize(v):
    '''
    Removes special characters and punctuation and makes all letters lowercase if value is a string
    '''
    if type(v) == str:
        return unidecode(v).translate(str.maketrans('', '', string.punctuation)).lower().strip()
    else:
        return v

## Standardizing strings

In [8]:
enderecos_15a = enderecos_15a.applymap(normalize)

In [9]:
enderecos_15a['logradouro'] = enderecos_15a['logradouro'].str.replace('rua','r')
enderecos_15a['logradouro'] = enderecos_15a['logradouro'].str.replace('avenida','av')

In [10]:
enderecos_dne = enderecos_dne.applymap(normalize)

# **Exploratory Analysis**

## Checking unique values of neighborhoods

In [22]:
enderecos_15a['bairro'].value_counts()

centro            6119
jatiuca           4990
ponta verde       4596
farol             4250
poco              3910
                  ... 
santo antao          1
santo cristo         1
brooklin novo        1
san martin           1
poco da panela       1
Name: bairro, Length: 455, dtype: int64

there is a lenght of 455 unique values, this is a much higher than expected, showing that there are spelling errors in filling in the neighborhoods field. 

Below we can visualize the less recurring values and better check these errors. 

### Visualizing the less recurring values

In [ ]:
b = enderecos_15a['bairro'].value_counts()
b[b < 5].index

Index(['jd petropolis', 'frances', 'gracas', 'magabeiras', 'tabuleiro novo',
       'vila yara', 'brooklin paulista', 'vergel', 'gruta', '0',
       ...
       'jardin petropolis', 'mutirao', 'benedito  bentes', 'ponta  da  terra',
       'tabuleior dos martins', 'horto florestal', 'benedito bentes  ll',
       'tabuleiro  dos  martins', 'benedito benstes',
       'residencial tabuleiro do martins i cidade universitaria'],
      dtype='object', length=358)

## Types of roads

We can do the same with types of roads

In [ ]:
enderecos_15a['logradouro'].str.split(expand=True)[0].value_counts()

rua              30674
av                8044
r                 7245
avenida           4012
avenidadr          577
                 ...  
aldebaran            1
ruadianapolis        1
rua5c                1
travessajuca         1
at                   1
Name: 0, Length: 1570, dtype: int64

# **Levenshtein Distance Correction**

To solve this problem, we can use the Levenshtein Distance to calculate the score of similarity  with referance database.

## Corrective Function

In [11]:
def match_names(name, list_names, min_score=0):
    return process.extractOne(name, list_names, scorer=fuzz.token_set_ratio)

### Neighborhood corretion

#### Replacing null values

In [12]:
enderecos_15a['bairro'] = enderecos_15a['bairro'].fillna('none')

#### Creating a list of unique values

In [13]:
bairros_15a = enderecos_15a['bairro'].unique().tolist()

#### Creating a list of correct neighborhoods to compare

In [14]:
bairros_dne = enderecos_dne['bairro'].unique().tolist()

#### List with matches +80

In [20]:
names = []
lower_matches = []
for x in bairros_15a:
    match = match_names(x, bairros_dne)
    if match[1] >= 80:
        name = (str(x), str(match[0]))
        names.append(name)
    else:
      lower_match = (str(x), str(match[0]), str(match[1]))
      lower_matches.append(lower_match)
bairro_dict= names
bairro_dict

[('centro', 'centro'),
 ('farol', 'farol'),
 ('bebedouro', 'cha de bebedouro'),
 ('pajucara', 'pajucara'),
 ('cruz das almas', 'cruz das almas'),
 ('jatiuca', 'jatiuca'),
 ('clima bom', 'clima bom'),
 ('santa amelia', 'santa amelia'),
 ('ponta verde', 'ponta verde'),
 ('tabuleiro do martins', 'tabuleiro do martins'),
 ('gruta de lourdes', 'gruta de lourdes'),
 ('barro duro', 'barro duro'),
 ('jaragua', 'jaragua'),
 ('tabuleiro dos martins', 'tabuleiro do martins'),
 ('serraria', 'serraria'),
 ('mangabeiras', 'mangabeiras'),
 ('jacintinho', 'jacintinho'),
 ('boa vista', 'boa vista'),
 ('garca torta', 'garca torta'),
 ('sao jorge', 'sao jorge'),
 ('ouro preto', 'ouro preto'),
 ('poco', 'poco'),
 ('tabulleiro dos martins', 'tabuleiro do martins'),
 ('levada', 'levada'),
 ('ponta grossa', 'ponta grossa'),
 ('pinheiro', 'pinheiro'),
 ('vergel do lago', 'vergel do lago'),
 ('tab do martins', 'tabuleiro do martins'),
 ('gruta de loudes', 'gruta de lourdes'),
 ('pontaverde', 'ponta verde'),
 (

#### List with low matches

In [16]:
lower_matches.sort()
lower_matches

[('0', 'zelia barbosa rocha', '0'),
 ('00114', 'zelia barbosa rocha', '0'),
 ('255', 'zelia barbosa rocha', '0'),
 ('57250300', 'zelia barbosa rocha', '0'),
 ('9', 'zelia barbosa rocha', '0'),
 ('adrianopolis', 'petropolis', '64'),
 ('aeroporto', 'ouro preto', '63'),
 ('aflitos', 'feitosa', '71'),
 ('afogados', 'eldorado', '50'),
 ('agua verde', 'sonho verde', '67'),
 ('aldeota', 'eldorado', '53'),
 ('alto da alegria', 'pontal da barra', '60'),
 ('alto da mooca', 'canto da saudade', '55'),
 ('alto do stella maris', 'alto do cruzeiro', '61'),
 ('anastacio', 'canafistula', '60'),
 ('antaresserraria', 'serraria', '70'),
 ('antonio lins de souza', 'padre antonio lima neto', '59'),
 ('arapirada', 'capiata', '62'),
 ('areao', 'paraiso', '67'),
 ('areia branca', 'santa amelia', '67'),
 ('areias', 'guaribas', '71'),
 ('asa sul', 'canafistula', '56'),
 ('av menino marcelo', 'tenorio cavalcante', '57'),
 ('baarra 2', 'baixa grande', '50'),
 ('bairro', 'baixao', '67'),
 ('bairro urbano', 'barro d

#### Percentage with matches +80

Now it is clear that the data is way more complicated than expected, only 59% of of the neighbohood had at last a score of 80 in Levensthein Distance. This indicates that this method is not suficient to an eficient corretion.

In [17]:
# Porcentagem de matches +80
(len(names)/len(bairros_15a))*100

58.901098901098905

#### Creating a dictionary with correct values

In [21]:
# Criando um dicionário que mapeia bairros com possíveis escritas e uma lista com valores sem mapeamento.
matches_dict = dict()
bairros_desconhecidos = list()

for name, match in bairro_dict:
  if match:
    if match in matches_dict:
      matches_dict[match].append(name)
    else:
      matches_dict[match] = list()
      matches_dict[match].append(name)
  else:
    if name != 'none':
      bairros_desconhecidos.append(name)

bairros_desconhecidos
matches_dict

{'alto do cruzeiro': ['alto do cruzeiro', 'cruzeiro'],
 'antares': ['antares',
  'antares i',
  'serraria  antares',
  'antares ii',
  'anteres'],
 'area rural de arapiraca': ['arapiraca'],
 'area rural de maceio': ['maceio'],
 'area rural de sao miguel dos campos': ['barra de sao miguel'],
 'baixa grande': ['baixa grande'],
 'baixao': ['baixao'],
 'barro duro': ['barro duro',
  'barro  duro',
  'borro duro',
  'serraria  barro duro',
  'baarro duro'],
 'bela vista': ['bela vista', 'parque bela vista'],
 'benedito bentes': ['benedito bentes',
  'conj benedito bentes i',
  'benedito bentes iii',
  'benedito bentes i',
  'benedito bentes ii',
  'b bentes',
  'benedito benstes',
  'bendito bentes i',
  'benedito  bentes',
  'bendito bentes',
  'benedito bestes',
  'benedito bentes 2',
  'benedito',
  'benedito bentes ll',
  'benedito bentes  ll'],
 'boa vista': ['boa vista'],
 'bom parto': ['bom parto', 'bom  parto'],
 'bom sucesso': ['bom sucesso'],
 'brasilia': ['brasilia'],
 'brasilian

#### Including correct values to original dataframe

In [ ]:
enderecos_15a['new_bairro'] = enderecos_15a['bairro'].replace(bairro_dict)
enderecos_15a

,numeroDocumentoPrincipal,nome,logradouro,numero,complemento,bairro,cidade,estado,pais,cep,new_bairro
0,bf580da36dfed6ee3721f0d26d63159014842b4c96b5b2...,770d60b893806689ec588a7bd1bfa5c7b3ee36645026e1...,NaN,NaN,NaN,none,maceio,NaN,brasil,NaN,none
1,7cb5d4bbb25f8d3a3cdcec95c1e88a264c62818cd9f782...,f3ecb0a4377b630243172484e0d01c2520c6a5426f060e...,r do comercio,181,edf banco economico 2o andar,centro,maceio,NaN,brasil,NaN,centro
2,e6dba38b2a567498a44c3e7d8aa6caff9d668381fd3e06...,05e2ce4d3f63e7005bdb772d65ac3378d0626808d0a254...,av fernandes lima,3349,NaN,farol,maceio,NaN,brasil,NaN,farol
3,0330aa7a12b5910df2712c58d667fb0fdb9b08b6bec5cf...,4428d9764b8a978655af3a268216981b505bb1a944bb03...,av fernandes limasn km 06,NaN,NaN,farol,maceio,NaN,brasil,57057900.0,farol
4,f3d81c1857f9bf4b3721645ef9e85c0a81043ce8e6a1a9...,224799ce270da8c15f18971a8fc8c507fc6359702b9451...,av da paz,21,lot leao irmaos,centro,maceio,NaN,brasil,57020440.0,centro
...,...,...,...,...,...,...,...,...,...,...,...
66856,e9b27803e549a9b5e69802ee55fe309ca33be671d966b0...,9e7f79bd4c7375a81f2dc8875829c4646984fc8b307862...,teonilo gama,430,NaN,trapiche da barra,maceio,NaN,brasil,57010384.0,trapiche da barra
66857,7d7513d9c1b2983bf40232039546912f4189dbcab47442...,20bc6700304dc43938f97b02791e6ae053d8a53e552dab...,r jangadeiros alagoanos,01188,sala 0801,pajucara,maceio,NaN,brasil,57.0,pajucara
66858,d7d06d824002fd4c9a556c25f97ba5ac6b31aa384f70cf...,b973036c61c1b4c0f4c5d901774950961a36eb246c3344...,r jangadeiros alagoanos,01188,sala 0801,pajucara,maceio,NaN,brasil,57.0,pajucara
66859,413c194c770b76e0ad2df6e86ed096a38fc42c56dea088...,f350137c9264596d55cfb6844dba28c53cd2bcb2e54340...,r jangadeiros alagoanos,01188,sala 0801,pajucara,maceio,NaN,brasil,57.0,pajucara
